# CONFLUENCE Tutorial: Point-Scale Workflow (Flux Tower Example)

This notebook demonstrates CONFLUENCE's simplest spatial configuration: point-scale modeling. We'll simulate vertical processes at a single location, typical for flux tower sites, weather stations, or snow monitoring locations.

## Overview

Point-scale modeling focuses on vertical processes without spatial heterogeneity:
- Energy balance
- Soil moisture dynamics  
- Snow accumulation and melt
- Evapotranspiration

## Learning Objectives

1. Understand point-scale modeling in CONFLUENCE
2. Configure CONFLUENCE for single-point simulations
3. Compare with spatially distributed approaches
4. Analyze point-scale model outputs

## 1. Understanding Point-Scale Modeling

Before we simulate entire watersheds, we need to understand the vertical processes at a single point.

### When to Use Point-Scale Modeling:

- **Flux tower validation**: Compare model outputs with eddy covariance measurements
- **Process understanding**: Study vertical water and energy fluxes without spatial complexity
- **Parameter calibration**: Optimize parameters at well-instrumented sites
- **Model development**: Test new parameterizations before scaling up
- **SNOTEL/Weather stations**: Validate snow accumulation and melt processes

In [1]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import contextily as cx
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Create Point-Scale Configuration

For point-scale modeling, we need to modify our configuration to:
1. Set `SPATIAL_MODE: Point`
2. Create a minimal domain (single point with small buffer)
3. Focus on vertical processes

We'll use a SNOTEL site as our example: Loveland Pass in Colorado.

In [2]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load template configuration
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'

# Read config file
with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update paths and settings for point-scale
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Modify for point-scale modeling
config_dict['DOMAIN_NAME'] = 'loveland_pass_snotel'
config_dict['EXPERIMENT_ID'] = 'point_scale_tutorial'
config_dict['SPATIAL_MODE'] = 'Point'  # Key setting for point-scale
config_dict['POUR_POINT_COORDS'] = '39.68/-105.88'  # Loveland Pass SNOTEL site

# Use lumped configuration (single unit)
config_dict['DOMAIN_DEFINITION_METHOD'] = 'lumped'
config_dict['DOMAIN_DISCRETIZATION'] = 'GRUs'

# Update experiment period for tutorial
config_dict['EXPERIMENT_TIME_START'] = '2020-10-01 00:00'
config_dict['EXPERIMENT_TIME_END'] = '2021-09-30 23:00'

# Set forcing data parameters
config_dict['FORCING_DATASET'] = 'ERA5'
config_dict['FORCING_VARIABLES'] = 'default'

# Save point-scale configuration to temporary file
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_notebook.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f)

print("=== Point-Scale Configuration ===")
print(f"Domain Name: {config_dict['DOMAIN_NAME']}")
print(f"Spatial Mode: {config_dict['SPATIAL_MODE']}")
print(f"Location: {config_dict['POUR_POINT_COORDS']}")
print(f"Period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"Forcing Data: {config_dict['FORCING_DATASET']}")

=== Point-Scale Configuration ===
Domain Name: loveland_pass_snotel
Spatial Mode: Point
Location: 39.68/-105.88
Period: 2020-10-01 00:00 to 2021-09-30 23:00
Forcing Data: ERA5


## 3. Initialize CONFLUENCE for Point-Scale

In [3]:
# Initialize CONFLUENCE with point-scale configuration
confluence = CONFLUENCE(temp_config_path)

print("=== CONFLUENCE Point-Scale Setup ===")
print(f"Project directory: {confluence.managers['project'].project_dir}")
print(f"Spatial mode: {confluence.config['SPATIAL_MODE']}")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")

16:17:55 - INFO - ============================================================


2025-05-11 16:17:55,931 - confluence_general - INFO - ============================================================


16:17:55 - INFO - CONFLUENCE Logging Initialized


2025-05-11 16:17:55,932 - confluence_general - INFO - CONFLUENCE Logging Initialized


16:17:55 - INFO - Domain: loveland_pass_snotel


...

16:17:55 - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


2025-05-11 16:17:55,975 - confluence_general - INFO - Initializing VariableHandler for dataset: ERA5 and model: SUMMA


16:17:56 - INFO - CONFLUENCE system initialized successfully


2025-05-11 16:17:56,310 - confluence_general - INFO - CONFLUENCE system initialized successfully


=== CONFLUENCE Point-Scale Setup ===
Project directory: /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel
Spatial mode: Point
Domain: loveland_pass_snotel


## 4. Setup Project Structure

In [4]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")
print("Creating point-scale project structure...")

# Setup project
project_dir = confluence.managers['project'].setup_project()

# Create pour point (in this case, our SNOTEL location)
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

print("\nNote: For point-scale modeling, we still use the same directory structure")
print("but the spatial extent is minimal (single point with small buffer).")

=== Step 1: Project Initialization ===
Creating point-scale project structure...
16:04:25 - INFO - Setting up project for domain: loveland_pass_snotel


2025-05-11 16:04:25,884 - confluence_general - INFO - Setting up project for domain: loveland_pass_snotel


16:04:25 - INFO - Updating bounding box for point-scale simulation to: 39.681/-105.881/39.679/-105.879


2025-05-11 16:04:25,982 - confluence_general - INFO - Updating bounding box for point-scale simulation to: 39.681/-105.881/39.679/-105.879


16:04:25 - WARNING - Active config file not found at /home/darri.eythorsson/code/CONFLUENCE/0_config_files/config_active.yaml


...

16:04:25 - INFO - Project directory created at: /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel


2025-05-11 16:04:25,987 - confluence_general - INFO - Project directory created at: /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel
2025-05-11 16:04:26,063 - pyogrio._io - INFO - Created 1 records


16:04:26 - INFO - Pour point shapefile created successfully: /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/shapefiles/pour_point/loveland_pass_snotel_pourPoint.shp


2025-05-11 16:04:26,067 - confluence_general - INFO - Pour point shapefile created successfully: /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/shapefiles/pour_point/loveland_pass_snotel_pourPoint.shp



Created directories:
  📁 _workLog_loveland_pass_snotel
  📁 attributes
  📁 documentation
  📁 observations
  📁 optimisation
  📁 shapefiles

Note: For point-scale modeling, we still use the same directory structure
but the spatial extent is minimal (single point with small buffer).


## 5. Geospatial Domain Definition - Data acquisition 

In [5]:
# Acquire attributes
print("Acquiring geospatial attributes for point location...")
print(f"Minimal bounding box: {confluence.config.get('BOUNDING_BOX_COORDS')}")
confluence.managers['data'].acquire_attributes()

Acquiring geospatial attributes for point location...
Minimal bounding box: 39.681/-105.881/39.679/-105.879
16:04:26 - INFO - Starting attribute acquisition


2025-05-11 16:04:26,086 - confluence_general - INFO - Starting attribute acquisition


16:04:26 - INFO - Acquiring elevation data


2025-05-11 16:04:26,101 - confluence_general - INFO - Acquiring elevation data


extract-gis.sh: WARNING! --account not provided, using `rpp-kshook` by default.
(2025-05-11 22:04:30) merit_hydro.sh: processing MERIT-Hydro GeoTIFF(s)...
(2025-05-11 22:04:30) merit_hydro.sh: creating cache directory under /home/darri.eythorsson/cache_loveland_pass_snotel
(2025-05-11 22:04:30) merit_hydro.sh: creating output directory under /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/attributes/elevation/dem
(2025-05-11 22:04:30) merit_hydro.sh: untarring MERIT-Hydro variables under /home/darri.eythorsson/cache_loveland_pass_snotel
(2025-05-11 22:06:02) merit_hydro.sh: subsetting GeoTIFFs under /work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/attributes/elevation/dem
(2025-05-11 22:06:02) merit_hydro.sh: deleting temporary files from /home/darri.eythorsson/cache_loveland_pass_snotel


...

mkdir: cannot create directory ‘/home/darri.eythorsson/empty_dir’: File exists
2025-05-11 16:07:27,520 - confluence_general - INFO - gistool completed successfully.


16:07:27 - INFO - Geospatial data acquisition process completed


2025-05-11 16:07:27,522 - confluence_general - INFO - Geospatial data acquisition process completed


16:07:27 - INFO - Attribute acquisition completed successfully


2025-05-11 16:07:27,523 - confluence_general - INFO - Attribute acquisition completed successfully


## 6. Geospatial Domain Definition - Domain creation

In [6]:
# Geospatial Domain Definition and Analysis
print("=== Step 2: Geospatial Domain Definition and Analysis ===")

# Define domain
print("\nDefining minimal domain for point-scale simulation...")
watershed_path = confluence.managers['domain'].define_domain()
# Discretize domain (single HRU for point-scale)
print("\nCreating single HRU for point-scale simulation...")
hru_path = confluence.managers['domain'].discretize_domain()

# Check outputs
print("\nDomain definition complete:")
print(f"  - Watershed defined: {watershed_path is not None}")
print(f"  - HRUs created: {hru_path is not None}")

# Verify single HRU
hru_dir = project_dir / 'shapefiles' / 'catchment'
if hru_dir.exists():
    hru_files = list(hru_dir.glob('*.shp'))
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        print(f"  - Number of HRUs: {len(hru_gdf)} (should be 1 for point-scale)")

=== Step 2: Geospatial Domain Definition and Analysis ===

Defining minimal domain for point-scale simulation...
16:07:27 - INFO - Domain definition workflow starting with: lumped


2025-05-11 16:07:27,534 - confluence_general - INFO - Domain definition workflow starting with: lumped


16:07:27 - INFO - Starting point buffer


2025-05-11 16:07:27,536 - confluence_general - INFO - Starting point buffer


16:07:27 - INFO - point buffer completed successfully


...

16:07:29 - INFO - Domain discretization completed in 1.27 seconds


2025-05-11 16:07:29,065 - confluence_general - INFO - Domain discretization completed in 1.27 seconds


16:07:29 - ERROR - Domain discretization failed - no shapefile created


2025-05-11 16:07:29,066 - confluence_general - ERROR - Domain discretization failed - no shapefile created



Domain definition complete:
  - Watershed defined: True
  - HRUs created: False
  - Number of HRUs: 1 (should be 1 for point-scale)


## 7. Model Agnostic Data Pre-Processing - Data Acquisition

In [7]:
# Step 3: Model Agnostic Data Pre-Processing
print("=== Step 3: Model Agnostic Data Pre-Processing ===")

# Process observed data (if we had real SNOTEL data, it would be processed here)
print("Processing observed data...")
# Note: We skip this for synthetic data, but normally would process SNOTEL data here

# Acquire forcings
print(f"\nAcquiring forcing data for point location...")
print(f"Dataset: {confluence.config['FORCING_DATASET']}")
confluence.managers['data'].acquire_forcings()

=== Step 3: Model Agnostic Data Pre-Processing ===
Processing observed data...

Acquiring forcing data for point location...
Dataset: ERA5
16:07:29 - INFO - Starting forcing data acquisition


2025-05-11 16:07:29,086 - confluence_general - INFO - Starting forcing data acquisition


16:07:32 - INFO - datatool job submitted successfully.


2025-05-11 16:07:32,674 - confluence_general - INFO - datatool job submitted successfully.


16:12:03 - INFO - All datatool array jobs completed.


2025-05-11 16:12:03,271 - confluence_general - INFO - All datatool array jobs completed.


16:12:03 - INFO - Meteorological data acquisition process completed


2025-05-11 16:12:03,275 - confluence_general - INFO - Meteorological data acquisition process completed


16:12:03 - INFO - Forcing data acquisition completed successfully


2025-05-11 16:12:03,277 - confluence_general - INFO - Forcing data acquisition completed successfully


## 8. Model Agnostic Data Pre-Processing - Remapping and zonal statistics

In [ ]:
# Run model-agnostic preprocessing
print("\nRunning model-agnostic preprocessing...")
print("For point-scale simulations:")
print("  - No spatial averaging needed")
print("  - Apply elevation corrections if needed")
print("  - Convert units to model requirements")
confluence.managers['data'].run_model_agnostic_preprocessing()

print("\nModel-agnostic preprocessing complete")


Running model-agnostic preprocessing...
For point-scale simulations:
  - No spatial averaging needed
  - Apply elevation corrections if needed
  - Convert units to model requirements
16:17:56 - INFO - Starting model-agnostic preprocessing


2025-05-11 16:17:56,320 - confluence_general - INFO - Starting model-agnostic preprocessing


16:17:56 - INFO - Running geospatial statistics


2025-05-11 16:17:56,323 - confluence_general - INFO - Running geospatial statistics


16:17:56 - INFO - Starting geospatial statistics calculation


...

EASYMORE saved target shapefile for EASYMORE claculation as:
/work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/forcing/temp_easymore_a5c87e96_0/loveland_pass_snotel_ERA5_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon


2025-05-11 16:19:31,043 - pyogrio._io - INFO - Created 4 records


/work/comphyd_lab/data/CONFLUENCE_data/domain_loveland_pass_snotel/forcing/temp_easymore_a5c87e96_0/loveland_pass_snotel_ERA5_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]


2025-05-11 16:19:31,120 - pyogrio._io - INFO - Created 1 records
2025-05-11 16:19:31,227 - pyogrio._io - INFO - Created 4 records
2025-05-11 16:19:31,310 - pyogrio._io - INFO - Created 1 records
2025-05-11 16:19:31,340 - pyogrio._io - INFO - Created 4 records
/work/comphyd_lab/users/darri/data/CONFLUENCE_data/installs/conf-env/lib/python3.11/site-packages/easymore/remapper.py:521: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/work/comphyd_lab/users/darri/data/CONFLUENCE_data/installs/conf-env/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'S_1_gru_to_seg' to 'S_1_gru_to'
  ogr_write(
/work/comphyd_lab/users/darri/data/CONFLUENCE_data/installs/conf-env/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'S_1_GRU_area' to 'S_1_GRU_ar'

Ended at date and time 2025-05-11 16:19:31.617835
It took 0.909169 seconds to finish creating of the remapping file
---------------------------
Running within a SLURM job.
SLURM environment variables found: ['SLURM_JOBID', 'SLURM_JOB_NAME', 'SLURM_NODELIST']
------REMAPPING------
netcdf output file will be compressed at level 4


## 9. Model Specific - Pre Processing 

In [ ]:
# Step 4: Model Specific Processing and Initialization
print("=== Step 4: Model Specific Processing and Initialization ===")

# Preprocess models
print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files for point-scale simulation...")
print("Point-scale specific settings:")
print("  - Single HRU configuration")
print("  - No lateral flow between units")
print("  - Focus on vertical processes")
confluence.managers['model'].preprocess_models()

## 10. Model Specific - Initialisation

In [ ]:
# Run models
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} for point-scale simulation...")
print("Point-scale models typically:")
print("  - Run faster than distributed models")
print("  - Focus on energy balance and vertical fluxes")
print("  - Are ideal for process-level validation")
confluence.managers['model'].run_models()

print("\nPoint-scale model run complete")
print("Note: Point-scale simulations typically don't require routing")

## 11. Calibration - Optional Steps

In [ ]:
# Optimization and Analysis)
print("=== Step 5 & 6: Optional Steps ===")

# For point-scale, we might focus on parameter calibration
if confluence.config.get('RUN_ITERATIVE_OPTIMISATION', False):
    print("Running point-scale parameter calibration...")
    optimization_results = confluence.managers['optimization'].calibrate_model()
else:
    print("Parameter calibration skipped")


## 12. Result Analysis and Evaluation

In [ ]:
# Step 7: Result Analysis and Evaluation
print("=== Step 7: Result Analysis and Evaluation ===")

# Postprocess results
print("Post-processing point-scale model results...")
confluence.managers['model'].postprocess_results()

# Get workflow status
status = confluence.workflow_orchestrator.get_workflow_status()
print(f"\nWorkflow Status:")
print(f"Total steps: {status['total_steps']}")
print(f"Completed steps: {status['completed_steps']}")

## Summary: Point-Scale Modeling Insights

### What We Accomplished:

1. **Set up point-scale simulation** for a SNOTEL site
2. **Minimal spatial domain** with single HRU
3. **Focused on vertical processes** without lateral flow
4. **Analyzed key variables** relevant to point observations

### Key Differences from Spatial Models:

- **No spatial heterogeneity**: Single point representation
- **No routing**: All processes occur at one location
- **Direct comparison**: Easy validation with point observations
- **Computational efficiency**: Fast execution for testing

### Applications of Point-Scale Modeling:

1. **Parameter calibration**: Optimize parameters at well-observed sites
2. **Process validation**: Test model physics against observations
3. **Model development**: Develop new parameterizations
4. **Sensitivity analysis**: Understand parameter influence

### Next Steps:

1. **Scale up to lumped basin**: Use calibrated parameters
2. **Compare multiple sites**: Test model transferability
3. **Ensemble simulations**: Explore parameter uncertainty
4. **Process experiments**: Test different model configurations